# Evaluation
To analyze the effects of the compression methods the tool provides a set of evaluation methods. 
This notebook demonstrates them.

## Set Up

Import dependencies

In [6]:
import sys
import torch
import importlib
import inspect
import torchvision.transforms as transforms
import torchvision.datasets as datasets

# Add thesis package to path
sys.path.append("../")

import src.general as general
import src.evaluation as eval

### Load Model

In [4]:
model_state = "../models/mnist.pt"
model_class = "models.mnist"

# Import the module classes
module = importlib.import_module(model_class)
classes = general.get_module_classes(module)
for cls in classes:
    globals()[cls.__name__] = cls

# Get device
device = general.get_device()

# Load the model
model = torch.load(model_state, map_location=torch.device(device))

Using cuda: False


### Load Dataset

In [7]:
# Load MNIST dataset
batch_size = 64

kwargs = {'num_workers': 1, 'pin_memory': True} if device == "cuda" else {}
mnist_transform = transform=transforms.ToTensor()

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True, transform=mnist_transform,),
    batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True, transform=mnist_transform,),
    batch_size=batch_size, shuffle=True, **kwargs),

## Metrics
Metrics can be loaded as follows.

In [8]:
import src.evaluation as evaluation

def get_metrics(model, example_input):
    model_size = evaluation.get_model_size(model)
    print("Model size (MB): {0:.2f}".format(model_size))

    model_parameters = evaluation.get_model_parameters(model)
    print("Model parameters: {0}".format(model_parameters))

    # Input size is necessary for FLOPS calculation, but is dependent on the dataset. 
    model_flops, params = evaluation.get_model_flops(model, example_input.size())
    print("Model FLOPS: {0}".format(model_flops))


input_batch = next(iter(train_loader))
example_input = input_batch[0][0]

get_metrics(model, example_input)

Model size (MB): 1.65
Model parameters: 431080
Model FLOPS: 2.30772M
